# 构造微调训练数据集

借助 ChatGPT 和 GPT API 我们可以实现自动化批量构造训练数据集。

下面我们以中国古典哲学数据集为例，展示了自动构造训练集的主要流程：

- 使用 LangChain 构造训练数据样例
    - 基于 ChatGPT 设计 `System Role` 提示词
    - 使用 `OpenAI GPT-3.5-Turbo-1106` 生成基础数据
    - 解析 OpenAI GPT 生成的训练数据
    - 持久化存储`dataset.csv`训练数据集文件
    - 使用 ChatGPT 实现训练数据多样化
- 自动化批量生成训练数据集
    - 整理收集原始数据`raw_data.txt`
    - 自动解析原始数据样例 `raw_data_content[]`
    - 设计 `gen_data` 训练数据生成器函数
    - 设计训练数据生成流水线

最佳实践参考：

- 使用 GPT-3.5 生成基础数据：https://platform.openai.com/playground/p/2c7XNPgo6Y2iDxILiWfD3iPu?model=gpt-3.5-turbo-1106&mode=chat
- 使用 ChatGPT 生成数据处理代码和相关文本整理：https://chat.openai.com/share/cdfd2d1d-a75e-4cee-be49-539c010ca1b1
- GPT API 价格: https://openai.com/pricing

## 使用 OpenAI SDk 构造训练数据

In [1]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
model_name = "gpt-3.5-turbo-16k"

In [2]:
response = client.chat.completions.create(
  model=model_name,
messages=[
    {
      "role": "system",
      "content": "你是中国古典哲学大师，尤其擅长周易的哲学解读。\n\n接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。\n\n示例输入：\n\n师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。\n师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。\n\n期待结果：\n\ncontent:\"师卦\"\nsummary:\"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。\n\n师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。\n\n师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。\""
    },
    {
      "role": "user",
      "content": "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"
    },
    {
      "role": "assistant",
      "content": "content:\"蒙卦\"\nsummary:\"在周易中，蒙卦象征着教育启蒙和智慧的初期阶段，由两个异卦相叠组成：下卦坎（水）和上卦艮（山）。坎象征水，艮象征山，代表了山下有水源，水流出山。这预示着在事物发展的起初阶段，人们会陷入蒙昧状态，因此教育成为当务之急。只有培养纯正无邪的品质，才能走上通达之路，如同泉水汇成江河。\n\n蒙卦的核心哲学是：事物发展的初期阶段必然处于蒙昧状态，但如果能把握时机，行动切合时宜，就能启蒙通达。蒙卦紧随始生卦《屯》之后，《序卦》中解释道：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这说明物在幼稚阶段都会经历蒙昧状态，找它蒙即是稚嫩。\n\n《象》中解释说：山下出泉，蒙；君子以果行育德，意味着像泉水从山下流出一样，启蒙教育有助于培育美德。\""
    },
    {
      "role": "user",
      "content": "需卦，本卦是异卦相叠，上卦为坎，下卦为乾。乾卦象征天空辽阔，坎卦象征乌云密布。雨水之于传统农业社会而言，是最大的需求，因此用这个意象来表现。古人引申为雨将下，万物皆待，所以叫需。\n需，古体字结构是“雨上天下”，从雨而声。如苍天下雨，滋养万物及人。从人的角度看，天下雨，则不易出行耕作，因此在家喝酒吃肉、静待天时。所以，这一卦的基本象征就是等待、不进，还有饮食之义。\n \n需卦位于蒙卦之后，《序卦》之中这样解释道：“物稚不可不养也，故受之以需。需者，饮食之道也。”有所需要，也是有所等待。\n《象》曰：云上于天，需;君子以饮食宴乐。\n《象》中这段话的意思是说：需卦的卦象是乾(天)下坎(水)上，为水在天上之表象。水汽聚集天上成为云层，密云満天，但还没有下雨，需要等待;君子在这个时候需要吃喝，饮酒作乐，即在等待的时候积蓄力量。\n需卦象征等待，启示人们守正待机的道理，属于中上卦。《象》这样评断此卦：明珠土埋日久深，无光无亮到如今，忽然大风吹土去，自然显露有重新。\n"
    },
    {
      "role": "assistant",
      "content": "content:\"需卦\"\nsummary:\"在周易中，需卦是一个富有深意的卦象，由两个异卦相叠组成：下卦乾（天）和上卦坎（水）。乾象征广阔的天空，而坎象征乌云密布，暗示着迫切需要雨水的意象。古人将雨水视为万物生长的必需，因此命名为“需”。需卦的基本象征是等待和饮食之道。\n\n需卦位于蒙卦之后，《序卦》中解释为“物稚不可不养也，故受之以需。需者，饮食之道也。”说明物在幼稚状态需要得到滋养，因此受到需卦的启示。《象》中描述：云上于天，需；君子以饮食宴乐。意味着天空乌云密布，需要等待雨水，而君子则在等待的过程中积蓄力量，享受饮食和欢乐。\n\n需卦的核心哲学是：等待，启示着守正待机的道理。它属于中上卦，象征着埋藏已久的明珠，经过大风吹去尘埃后，重新显露光芒。\""
    }
  ],
  temperature=1,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [3]:
response

ChatCompletion(id='chatcmpl-NiCEupUgBpBxXnDOBlu8TmGNzvCaq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='由于需要按照格式要求进行输入，我将整理后的内容进行格式化输出：  \n\n```json\n{\n    "content": "需卦",\n    "summary": "在周易中，需卦是一个富有深意的卦象，由两个异卦相叠组成：下卦乾（天）和上卦坎（水）。乾象征广阔的天空，而坎象征乌云密布，暗示着迫切需要雨水的意象。古人将雨水视为万物生长的必需，因此命名为“需”。需卦的基本象征是等待和饮食之道。\\n\\n需卦位于蒙卦之后，《序卦》中解释为“物稚不可不养也，故受之以需。需者，饮食之道也。”说明物在幼稚状态需要得到滋养，因此受到需卦的启示。《象》中描述：云上于天，需；君子以饮食宴乐。意味着天空乌云密布，需要等待雨水，而君子则在等待的过程中积蓄力量，享受饮食和欢乐。\\n\\n需卦的核心哲学是：等待，启示着守正待机的道理。它属于中上卦，象征着埋藏已久的明珠，经过大风吹去尘埃后，重新显露光芒。"\n}\n```  ', role='assistant', function_call=None, tool_calls=None))], created=1722743717, model='gpt-3.5-turbo-16k', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=429, prompt_tokens=2472, total_tokens=2901))

## 使用 LangChain 构造训练数据

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

chat = ChatOpenAI(
    model=model_name,
    temperature=1,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

E:\Code\ai-translator-glm\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intende

In [5]:
system_content = """
你是中国古典哲学大师，尤其擅长周易的哲学解读。

接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

示例输入：

师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

期待结果：

content:"师卦"
summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
"""

In [6]:
# 原始数据
raw_content = "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"

In [7]:
messages = [
    SystemMessage(
        content=system_content
    ),
    HumanMessage(
        content=raw_content
    ),
]

In [8]:
ai_message = chat(messages)

E:\Code\ai-translator-glm\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


### 解析 OpenAI GPT 生成的训练数据

In [9]:
ai_message.content

'content:"蒙卦"\nsummary:"蒙卦是周易中一个富有智慧的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦艮（山）。这一卦象象征着教育的启蒙和智慧的诞生。在这里，艮卦象征着山，暗示停止不前；而坎卦象征着泉水，意味着从山下涌出的泉水。正如泉水汇聚成江河一样，蒙昧的时期会逐渐被启蒙所破除。\n\n蒙卦传达的核心哲学是：事物在发展的初期阶段通常是蒙昧的，故而教育是至关重要的。教育可以培养学生的纯正品质，使他们渐渐开启智慧之门。蒙卦提醒人们应当把握时机，适时采取行动，这样才能达到启蒙和通达的目的。\n\n蒙卦紧随始生卦屯卦之后，《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”意即在事物刚刚开始的阶段，往往是蒙昧未开的状态，就像婴儿蒙昧未开。而《象》中解释蒙卦的意义是“山下出泉，蒙；君子以果行育德”，启示人们应当在蒙昧的状态下果断行动，培养德行。"'

In [10]:
text = ai_message.content

# 分割字符串来找到content和summary的位置
content_start = text.find('content:"') + len('content:"')
content_end = text.find('"\nsummary:')
summary_start = text.find('summary:"') + len('summary:"')
summary_end = text.rfind('"')

# 提取并存储content和summary
content = text[content_start:content_end].strip()
summary = text[summary_start:summary_end].strip()

print("Content:", content)
print("Summary:", summary)


Content: 蒙卦
Summary: 蒙卦是周易中一个富有智慧的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦艮（山）。这一卦象象征着教育的启蒙和智慧的诞生。在这里，艮卦象征着山，暗示停止不前；而坎卦象征着泉水，意味着从山下涌出的泉水。正如泉水汇聚成江河一样，蒙昧的时期会逐渐被启蒙所破除。

蒙卦传达的核心哲学是：事物在发展的初期阶段通常是蒙昧的，故而教育是至关重要的。教育可以培养学生的纯正品质，使他们渐渐开启智慧之门。蒙卦提醒人们应当把握时机，适时采取行动，这样才能达到启蒙和通达的目的。

蒙卦紧随始生卦屯卦之后，《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”意即在事物刚刚开始的阶段，往往是蒙昧未开的状态，就像婴儿蒙昧未开。而《象》中解释蒙卦的意义是“山下出泉，蒙；君子以果行育德”，启示人们应当在蒙昧的状态下果断行动，培养德行。


### 持久化存储训练数据集文件

In [11]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

# 新建CSV文件并写入数据
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 写入标题行
    writer.writerow(['content', 'summary'])
    # 写入数据行
    writer.writerow([content, summary])


### 数据增强：构造多样化的提问方式

In [12]:
def generate_question_summary_pairs(content, summary):
    """
    生成20对提问和总结的配对。

    :param content: 内容（例如：“蒙卦”）。
    :param summary: 内容的总结。
    :return: 包含20对提问和总结的列表。
    """
    # 20种提问模板
    question_templates = [
        "{}代表什么？",
        "周易中的{}含义是什么？",
        "请解释一下{}。",
        "{}在周易中是什么象征？",
        "周易{}的深层含义是什么？",
        "{}和教育启蒙有什么联系？",
        "周易的{}讲述了什么？",
        "{}是怎样的一个卦象？",
        "{}在周易中怎样表达教育的概念？",
        "{}的基本意义是什么？",
        "周易中{}的解释是什么？",
        "{}在周易中代表了哪些方面？",
        "{}涉及哪些哲学思想？",
        "周易中{}的象征意义是什么？",
        "{}的主要讲述内容是什么？",
        "周易{}的核心思想是什么？",
        "{}和启蒙教育之间有何联系？",
        "在周易中，{}象征着什么？",
        "请描述{}的含义。",
        "{}在周易哲学中扮演什么角色？"
    ]

    # 使用content填充提问模板
    questions = [template.format(content) for template in question_templates]

    # 创建提问和总结的配对
    question_summary_pairs = [(question, summary) for question in questions]

    return question_summary_pairs

In [13]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
pairs = generate_question_summary_pairs(content, summary)

# 将结果写入CSV文件
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['content', 'summary'])
    for pair in pairs:
        writer.writerow(pair)

## 自动化批量生成训练数据流水线

原始数据来源：https://www.zhouyi.cc/zhouyi/yijing64/4103.html

In [14]:
# 初始化一个空列表用于存储原始内容数据
raw_content_data = []

# 读取文件并分割数据样例
with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    # 使用连续的换行符('\n\n')作为分隔符来分割文本
    data_samples = content.split('\n\n')

    # 遍历分割后的数据样例并添加到列表中
    for sample in data_samples:
        # 移除每个样例中的额外空白字符（如果有的话）
        cleaned_sample = sample.strip()
        # 仅添加非空样例
        if cleaned_sample:
            raw_content_data.append(cleaned_sample)

In [15]:
# 输出结果以验证
for i, sample in enumerate(raw_content_data[:5]):  # 打印前5个样例以检查
    print(f"样例 {i+1}:")
    print(sample)
    print("------")


样例 1:
蒙卦原文
蒙。亨。匪我求童蒙，童蒙求我。初筮告，再三渎，渎则不告。利贞。
象曰：山下出泉，蒙。君子以果行育德。
白话文解释
蒙卦：通泰。不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我。第一次占筮，神灵告诉了他。轻慢不敬的再三占筮，轻慢不敬的占筮，神灵就不会告诉他。但还是吉利的卜问。
《象辞》说：上卦为艮，象征山；下卦为坎，象征泉。山下有泉，泉水喷涌而出，这是蒙卦的卦象。君子观此卦象，取法于一往无前的山泉，从而以果敢坚毅的行动来培养自身的品德。
《断易天机》解
蒙卦艮上坎下，为离宫四世卦。蒙即蒙昧，主回还往复，疑惑不前，多忧愁过失，乃是凶卦。
北宋易学家邵雍解
智慧未开，蒙昧闭塞；犹豫不决，缺乏果断。
得此卦者，智慧犹如童蒙，不辨是非，迷失方向；若能顺贤师良友之教，启其聪明则亨通。
台湾国学大儒傅佩荣解
时运：蓄积德行，出而用世。
财运：矿山生意，果决则吉。
家宅：君子居吉；婚姻之始。
身体：驱去邪热，可保平安。
传统解卦
这个卦是异卦（下坎上艮）相叠，艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜，因此，具有启蒙和通达的卦象。
大象：蒙者，昏而无所见也，故宜「启蒙」。
运势：初时迷惑不知方向，须忍耐待机而动，凡事多听取别人意见，则运可通。
事业：事业开始，混乱无序，危机四伏，以勇敢坚毅的行动可以扭转局面。然而必须接受严格教育，培养这种奋发图强的精神。务必脚踏实地，最忌好高骛远，否则会陷入孤立无援的境地。
经商：务必小心谨慎，不得急功近利，尤其应树立高尚的商业道德，以良好的信誉提高竞争力而取胜。
求名：必须接受良好的基础教育，陶冶情操。且动机纯正，可以达到目的。
婚恋：注意考察对方品德，不可以金钱为诱铒。夫妻需相互宽容、理解。
决策：有时会陷入迷惘困顿的境地，加上胆小、不果断，往往误事。如能接受长辈的教诲，甚至严酷的考验，抛弃疑惧的心理，等待适当时机，必然一帆风顺。
------
样例 2:
屯卦原文
屯。元，亨，利，贞。勿用，有攸往，利建侯。
象曰：云，雷，屯；君子以经纶。
白话文解释
屯卦。大吉大利，吉利的占卜。不利于出门。有利于建国封侯。
《象辞》说：屯的上卦为坎，坎为云，下卦为震，震为雷。云行于上，雷动于下，是屯卦的卦象。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国

### 将以上的所有模块，整合到一起，自动化生成数据

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

# 初始化LangChain的GPT-3.5调用
chat = ChatOpenAI(model=model_name,
                  temperature=1,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0)

def gen_data(raw_content):
    """
    使用LangChain GPT-3.5调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-3.5模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


In [17]:
# 示例调用（使用 raw_data.txt 中解析的数据样例）
generated_content = gen_data(raw_content_data[0])
print(generated_content)

content:"蒙卦"
summary:"蒙卦是一卦象，代表通泰。在这个卦象中，坎下艮上，象征着山下有泉水涌出，暗示蒙昧之中也包含着启蒙的可能。君子观此卦象，以山泉果敢的行动来培养自身的品德。蒙卦揭示了迷惑不前的困境，需要等待时机，多听取他人意见，方能通达。但如果轻慢不敬、犹豫不决，将导致困惑和过失，是凶兆。因此，必须接受教诲，培养果断精神，时机成熟时再行动，方能顺利。"
interpretation:"蒙卦是山下有泉的卦象，启示着在迷惑之中也蕴含启蒙的可能。蒙意味着迷茫无知，需要借助他人的智慧和指引才能通达。君子观此卦象，应当以果敢坚毅的行动来培养自身的品德。蒙卦警示着在困境中需要耐心等待时机，受教育启蒙，遵循行动的时机，方能转危为安。轻慢不敬、犹豫不决将导致困惑和过失，必须接受恰当的教诲才能化解困境，培养果断的品质和行动力，方能突破困境，通向成功。"
fortune_telling:"蒙卦初时象征迷惑不知方向，需要忍耐待机而动，多听取别人意见有助于通达。事业初起阶段可能混乱无序，但以勇敢坚毅的行动可以扭转局面。但务必接受严格教育，培养奋发图强的精神。经商宜小心谨慎、树立商业道德。求名应接受基础教育，注重品德，可达目的。婚恋需审慎考虑对方品德，建立互相宽容理解的关系。在决策时遇到困难时应接受教诲、等待适当时机，可获顺利。"
trad_consideration:"蒙卦是山下有泉的卦象，暗示着启蒙和通达。事业初始阶段可能会有混乱和危机，需要勇敢果断的行动才能扭转局面。务必注重教育培养，不可好高骛远，须踏实前行。经商应慎重，树立信誉取胜。求名应接受基础教育，培养高尚品德。婚姻需谨慎选择，相互包容理解。决策时遭遇困难应接受教诲，等待时机再行动，方能顺利。"
interpretation_example:"蒙卦启示着在困惑中也蕴涵着启蒙的可能。正如山下有泉水涌出一样，表明在蒙昧迷惑的状态中也存有通达的机会。君子观此卦象，应当以果敢坚毅的行为来塑造自己的品德，正如山泉果敢涌流。因此，蒙卦提醒人们在困境中需要等待正确时机，多听取他人意见，接受教诲，培养果断的品质，方能突破困境，通往成功之路。"


In [18]:
def dataset_parser(ai_message_content):
    """
    解析由gen_data函数生成的ai_message.content，提取content和summary。

    :param ai_message_content: gen_data函数返回的文本。
    :return: 提取的content和summary。
    """
    # 分割字符串来找到content和summary的位置
    content_start = ai_message_content.find('content:"') + len('content:"')
    content_end = ai_message_content.find('"\nsummary:')
    summary_start = ai_message_content.find('summary:"') + len('summary:"')
    summary_end = ai_message_content.rfind('"')

    # 提取并存储content和summary
    content = ai_message_content[content_start:content_end].strip()
    summary = ai_message_content[summary_start:summary_end].strip()

    return content, summary


In [19]:
# 示例调用（使用假设的gen_data函数返回的文本）
content, summary = dataset_parser(generated_content)
print("Content:", content)
print("Summary:", summary)

Content: 蒙卦
Summary: 蒙卦是一卦象，代表通泰。在这个卦象中，坎下艮上，象征着山下有泉水涌出，暗示蒙昧之中也包含着启蒙的可能。君子观此卦象，以山泉果敢的行动来培养自身的品德。蒙卦揭示了迷惑不前的困境，需要等待时机，多听取他人意见，方能通达。但如果轻慢不敬、犹豫不决，将导致困惑和过失，是凶兆。因此，必须接受教诲，培养果断精神，时机成熟时再行动，方能顺利。"
interpretation:"蒙卦是山下有泉的卦象，启示着在迷惑之中也蕴含启蒙的可能。蒙意味着迷茫无知，需要借助他人的智慧和指引才能通达。君子观此卦象，应当以果敢坚毅的行动来培养自身的品德。蒙卦警示着在困境中需要耐心等待时机，受教育启蒙，遵循行动的时机，方能转危为安。轻慢不敬、犹豫不决将导致困惑和过失，必须接受恰当的教诲才能化解困境，培养果断的品质和行动力，方能突破困境，通向成功。"
fortune_telling:"蒙卦初时象征迷惑不知方向，需要忍耐待机而动，多听取别人意见有助于通达。事业初起阶段可能混乱无序，但以勇敢坚毅的行动可以扭转局面。但务必接受严格教育，培养奋发图强的精神。经商宜小心谨慎、树立商业道德。求名应接受基础教育，注重品德，可达目的。婚恋需审慎考虑对方品德，建立互相宽容理解的关系。在决策时遇到困难时应接受教诲、等待适当时机，可获顺利。"
trad_consideration:"蒙卦是山下有泉的卦象，暗示着启蒙和通达。事业初始阶段可能会有混乱和危机，需要勇敢果断的行动才能扭转局面。务必注重教育培养，不可好高骛远，须踏实前行。经商应慎重，树立信誉取胜。求名应接受基础教育，培养高尚品德。婚姻需谨慎选择，相互包容理解。决策时遭遇困难应接受教诲，等待时机再行动，方能顺利。"
interpretation_example:"蒙卦启示着在困惑中也蕴涵着启蒙的可能。正如山下有泉水涌出一样，表明在蒙昧迷惑的状态中也存有通达的机会。君子观此卦象，应当以果敢坚毅的行为来塑造自己的品德，正如山泉果敢涌流。因此，蒙卦提醒人们在困境中需要等待正确时机，多听取他人意见，接受教诲，培养果断的品质，方能突破困境，通往成功之路。


In [20]:
import csv
import datetime
import os
from tqdm import tqdm

def main():
    # 确保 data 目录存在
    if not os.path.exists('data'):
        os.makedirs('data')

    # 解析 data/raw_data.txt 得到 raw_content_data 列表
    raw_content_data = []
    with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
        content = file.read()
        data_samples = content.split('\n\n')
        for sample in data_samples:
            cleaned_sample = sample.strip()
            if cleaned_sample:
                raw_content_data.append(cleaned_sample)
    print(f"加载raw_content: {len(raw_content_data)}条")
    # 创建带有时间戳的CSV文件名
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/zhouyi_dataset_{timestamp}.csv"
    print(filename)

    # 创建CSV文件并写入标题行
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['content', 'summary'])

        # 循环遍历 raw_content_data 数据样例
        for raw_content in tqdm(raw_content_data):
            # 调用 gen_data 方法得到 ai_message_content
            try:
                ai_message_content = gen_data(raw_content)

                # 解析 ai_message_content 得到 content 和 summary
                content, summary = dataset_parser(ai_message_content)
                
                print("Content:", content)
                print("Summary:", summary)
    
                # 调用 generate_question_summary_pairs 得到20组 pairs
                pairs = generate_question_summary_pairs(content, summary)
    
                # 将 pairs 写入 csv 文件
                for pair in pairs:
                    writer.writerow(pair)
            except Exception as e:
                print(str(e))


In [21]:
# 执行主函数
main()

加载raw_content: 24条
data/zhouyi_dataset_20240804_115641.csv


  4%|▍         | 1/24 [00:04<01:49,  4.77s/it]

Content: 相叠，下卦为坎，上卦为艮。蒙卦象征着幼稚愚昧之人的求教和顾问。在初次占筮时，神灵会告知真相。但如果不敬轻慢地多次占筮，神灵就不会告诉他。然而，蒙卦仍然是吉利的贞卜。

蒙卦的核心思想是，虽然刚开始可能会犹豫不决，迷失方向，但只要抓住时机，恰当行动，就能够启蒙通达。就像山下有泉水喷涌而出一样，君子应该以果断坚毅的行动来培养自身的品德。

在运势上，蒙卦表示初时可能会迷惑不清，不知道应该选择何方，因此需要忍耐等待良机。在事业上，蒙卦代表事业初创阶段的混乱与危机，但通过勇敢坚毅的行动可以扭转局面。在经商方面，必须小心谨慎，树立良好的商业道德，并建立稳定的商业信誉。在求名方面，必须接受良好的基础教育，培养自身的素质，才能够达到目标。在婚恋方面，需要考察对方的品德，注重相互宽容理解。

总之，蒙卦是一个需要勇气和耐心的阶段，通过正确的行动和良好的教育引导，可以实现顺利通达的结果
Summary: 相叠，下卦为坎，上卦为艮。蒙卦象征着幼稚愚昧之人的求教和顾问。在初次占筮时，神灵会告知真相。但如果不敬轻慢地多次占筮，神灵就不会告诉他。然而，蒙卦仍然是吉利的贞卜。

蒙卦的核心思想是，虽然刚开始可能会犹豫不决，迷失方向，但只要抓住时机，恰当行动，就能够启蒙通达。就像山下有泉水喷涌而出一样，君子应该以果断坚毅的行动来培养自身的品德。

在运势上，蒙卦表示初时可能会迷惑不清，不知道应该选择何方，因此需要忍耐等待良机。在事业上，蒙卦代表事业初创阶段的混乱与危机，但通过勇敢坚毅的行动可以扭转局面。在经商方面，必须小心谨慎，树立良好的商业道德，并建立稳定的商业信誉。在求名方面，必须接受良好的基础教育，培养自身的素质，才能够达到目标。在婚恋方面，需要考察对方的品德，注重相互宽容理解。

总之，蒙卦是一个需要勇气和耐心的阶段，通过正确的行动和良好的教育引导，可以实现顺利通达的结果


  8%|▊         | 2/24 [00:11<02:11,  6.00s/it]

Content: "屯卦"
summary："屯卦"是一个由坎卦（水）下震卦（雷）上叠合而成的异卦，象征着艰难的开始。大凶大吉，吉利的预兆，但不利于出门，有利于建立国家封侯。

在《象辞》中，上卦坎为云，下卦震为雷。云行于上，雷动于下，寓意着困境中的艰难险阻，需要坚毅果敢才能获得吉祥。君子观此卦象，取法于云雷，使用云的恩泽和雷的威严来治理国事。

根据《断易天机》，屯卦代表有困难，需要果敢勇敢面对危险境地才能取得吉利。古代北宋易学家邵雍解释，万物始生，初始困难，先劳后逸，须经过努力和坚持才能获得甘美果实。

傅佩荣先生的解析中指出，时机适宜守而不宜进，财运创业颇艰，家宅宜修缮或注意初婚关系，身体需注意保存元气。

屯卦的传统解卦认为，虽然初时困难重重，但要勇往直前，灵活应对，随机应变，方可迎来大的成功。起初多有不顺，但随着坚忍不拔的努力，困难将逐渐被克服，带来成功。

在事业上，需要克服困难，坚定信念，勇往直前。经商初期难免艰难，但不可放弃，应稳扎稳打，积极进取。在婚恋方面，不忘初心，坚守忠贞，勇敢追求，婚姻将会美满。

总的来说，屯卦预示着初始困难，但只要坚持不懈，克服艰难，积极应对，最终将获得成功。算命求名宜积极进取，能够成功，但需要注意困境带来的孤独和压力，需要依靠他人的帮助。展望未来，乐观向上，战胜困难，才能取得辉煌成就
Summary: "屯卦"
summary："屯卦


...

Content: 剥卦
Summary: 在周易中，剥卦是一个具有深意的卦象，由两个异卦相叠组成：下卦坤（地）和上卦艮（山）。这一卦象象征着剥落，表示山体高耸而朝向地面斜坡，风雨侵蚀剥落，暗示着困境的到来。

观察山石剥落，岩角崩塌，君子应当戒备，并以厚结民心，使人民安居乐业。

剥卦核心哲学是：在阴气回升的时候，阳气衰弱。小人得势，君子困顿，有利与不利，时运的转变带来失败和挫折。

剥卦运势不佳，此时应重新调整布局，不宜冒险行事。谨慎忍耐，静观其变，以期望将来时运的好转。在事业上，顺势而为，增强实力，等待时机的到来。

剥卦的象征意义体现在经商、求名和婚恋等方面，建议谨慎考虑，并注重家庭和睦。

对于一个人一生的决策，剥卦提示我们可能会遇到曲折、困境甚至绝境，但只要冷静面对，顽强拼搏，适应时势，并防范小人陷害，将会迎来命运的转变和美好的结果。关键是不要冒险行事，顺应时势。


 96%|█████████▌| 23/24 [02:34<00:07,  7.24s/it]

Content: "坎卦
Summary: "坎卦"
summary: "坎卦是一个同卦相叠的卦象，下卦上卦皆为坎。坎代表着险难和困境，象征着水流不息、险阻重重的境况。在这个卦象中，人们面临着重重困难和危机，必须保持信心和冷静才能够顺利度过困境。

坎卦的核心思想是：面对重重险阻，只有坚定信心才能通往成功的道路。尽管处境艰难，但只要保持诚信、沉着冷静，并努力创造条件改变现状，即可化险为夷，转危为安。同时，务必保持谨慎态度，不与人争执，静观时机，运用智慧突破困局。

坎卦适用于各个方面的解析，包括事业、经商、求名、婚恋等。在决策时，坎卦提示要保持信心，以沉着、不慌不忙的态度面对困境，勿失正常，方可转危为安，取得成功。"

interpretation: "《坎卦》描绘了一幅险阻重重的境况，处于重重困难危机之中。卦象中阳气虚弱，阴气强盛，需要坚定信心才能冲破困境。这个时候不宜冒险，也不可束手待毙，要以谨慎态度面对挑战，保持诚信和冷静。只有如此，才能以智慧和勇气化解险境，实现转危为安的目标。"

historical_interpretation: "坎卦在历史上被解释为困难重重，险境丛生，需要谨慎行事。北宋易学家邵雍认为，得到坎卦的人可能会面临运气不佳、事多困阻等挑战，需要退守保安，避免冒险。台湾国学大儒傅佩荣则指出，时运逐步升迁，但随时需要防患。财运如流水，务必谨言慎行，顺势而为。"

traditional_meaning: "根据传统解卦，坎卦象征着转运前的考验，虽面临困境和危机，但只要保持冷静、不轻举妄动，坚守信念，洞察时机，便能转危为安，化解险境。在事业方面，要努力创造机会，克服困难，不可心急，避免争执。在经商上，应保持头脑清醒，不急于求成，把握时机。在婚恋关系中，双方应共同克服困难，互相理解，共同前行。"

具体含义：
- 事业：面临困境和挑战，需要坚持信念，不轻言放弃，努力创造转机，保持冷静。
- 经商：不利之际，需谨慎行事，保持清醒头脑，把握时机，脱离困境。
- 求名：困难时期，需要坚守初衷，循序渐进，克服挫折，等待机遇到来。
- 婚恋：遭遇困难，双方要携手共渡难关，相互支持，共同努力。
- 决策：转运前的考验，保持信心，谨慎行事，不冒险，不轻举妄动，等待时机转化困境。


100%|██████████| 24/24 [02:38<00:00,  6.60s/it]

Content: 渐卦
Summary: 在周易中，渐卦代表逐渐进步的过程，由上卦巽（木）和下卦艮（山）组成。象征着山上有树木不断生长的情景，反映了循序渐进的发展态势。渐卦启示君子应该以贤德自居，担负起改善社会风俗的责任。

渐卦的核心哲学是循序渐进，积少成多。只有稳步前行，不急于求成，方能逐渐走向光明前途。在经商求名方面，亦应量力而行，坚守商业道德，努力提升自身修养和能力，方能取得成功。

渐卦所代表的时运是逐渐转好，财运上也会逐渐得利。在家宅和身体方面，需要安居调养，不宜急躁冒进。

传统解卦认为，渐卦预示着事业、婚恋等方面的逐渐进步，但需注意避免性情急躁、行事急进的情况。


### 异常分析


训练第一个 epoch 时，Training Loss 比较奇怪：

```
Step	Training Loss
1	3.594100
2	4.049100
3	3.091200
4	3.381700
5	3.547800
6	2.610200
7	2.657900
8	3.163900
```

通过解析 GPT-3.5-Turbo-1106 生成结果发现问题

In [ ]:
def gen_data(raw_content):
    """
    使用LangChain GPT-3.5调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-3.5模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

        返回格式要求：
        content:"{卦名}"
        summary:"{内容}"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

In [ ]:
# 执行主函数
main()